In [ ]:
# %pip install datasets
# from datasets import load_dataset

# # Load the Amazon Reviews 2023 dataset
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Home_and_Kitchen", trust_remote_code=True)

# # Print the dataset splits and details
# print(dataset)


In [ ]:
# dataset.save_to_disk("amazon_reviews_dataset")

In [ ]:
# import os
# from google.cloud import storage

# # Set your project and bucket name
# bucket_name = 'recomviz_home_and_kitchen'
# destination_folder = 'datasets/amazon_reviews_dataset'  # GCS path

# # Initialize storage client
# client = storage.Client()
# bucket = client.bucket(bucket_name)

# # Upload function
# def upload_folder_to_gcs(local_path, gcs_path):
#     for root, _, files in os.walk(local_path):
#         for file in files:
#             local_file = os.path.join(root, file)
#             relative_path = os.path.relpath(local_file, local_path)
#             blob_path = os.path.join(gcs_path, relative_path)
#             blob = bucket.blob(blob_path)
#             blob.upload_from_filename(local_file)
#             print(f"Uploaded {local_file} to {blob_path}")

# # Call the function
# upload_folder_to_gcs("amazon_reviews_dataset", destination_folder)

In [ ]:
# %pip install google-cloud-storage
# from google.cloud import storage
# import os

# def download_gcs_folder(bucket_name, gcs_folder, local_folder):
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)

#     blobs = bucket.list_blobs(prefix=gcs_folder)

#     for blob in blobs:
#         # Get local path
#         local_path = os.path.join(local_folder, os.path.relpath(blob.name, gcs_folder))

#         # Create local directories if needed
#         os.makedirs(os.path.dirname(local_path), exist_ok=True)

#         # Download the file
#         blob.download_to_filename(local_path)
#         print(f"Downloaded {blob.name} to {local_path}")

# # 🔁 Replace with your values:
# bucket_name = "recomviz_home_and_kitchen"
# gcs_folder = "datasets/amazon_reviews_dataset"  # GCS path (folder)
# local_folder = "amazon_reviews_dataset"          # Local target

# download_gcs_folder(bucket_name, gcs_folder, local_folder)

In [1]:
# %pip install datasets
import os
# %pip install google-cloud-storage
from google.cloud import storage
from datasets import load_from_disk

# Make sure the path is correct for your dataset location on disk
dataset = load_from_disk("amazon_reviews_dataset")
print(dataset)

Loading dataset from disk:   0%|          | 0/45 [00:00<?, ?it/s]

DatasetDict({
    full: Dataset({
        features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
        num_rows: 67409944
    })
})


In [2]:
user_item_dataset = dataset["full"].select_columns(["rating", "parent_asin", "user_id"])
print(user_item_dataset)

Dataset({
    features: ['rating', 'parent_asin', 'user_id'],
    num_rows: 67409944
})


In [5]:
from datasets import load_from_disk
import pandas as pd
from tqdm import tqdm

In [6]:
batch_size = 100_000
total_rows = user_item_dataset.num_rows
bucket_path = "gs://recomviz_home_and_kitchen/datasets/converted_user_ratings"

In [7]:
# Batch conversion with progress bar
for i in tqdm(range(0, total_rows, batch_size), desc="Uploading user-item ratings in batches"):
    # Select batch
    batch = user_item_dataset.select(range(i, min(i + batch_size, total_rows)))
    df = batch.to_pandas()
    
    # Drop rows with nulls
    df = df.dropna(subset=["user_id", "parent_asin", "rating"])
    
    # Rename for consistency
    df.rename(columns={"parent_asin": "product_id"}, inplace=True)
    
    # Save batch as Parquet to GCS
    df.to_parquet(
        f"{bucket_path}/chunk_{i//batch_size:05}.parquet",
        index=False,
        engine="pyarrow",
        storage_options={"token": "cloud"}  # Works in Vertex AI
    )

Uploading user-item ratings in batches: 100%|██████████| 675/675 [04:11<00:00,  2.68it/s]
